<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/1_gen_alpha_emitters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import astropy.units as units
import astropy.constants as const
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import IPython
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
import urllib.request

def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [2]:
df = lc_read_csv(livechart + 
"fields=decay_rads&nuclides=all&rad_types=a")
df = df[df["decay_1"] == "A"]
df

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift,energy,unc_e,...,unc_sp,binding,unc_ba,atomic_mass,unc_am,massexcess,unc_me,ENSDFpublicationcut-off,ENSDFauthors,Extraction_date
27,4,2,Be,,NaN,,NaN,,0,,...,50,4487.2478,9080,6019726.409,5848,18375.034,5448,23-Aug-2001,HU and TILLEY and KELLEY et al.,2022-04-26
29,4,4,Be,,NaN,,NaN,,0,,...,356,7062.4356,44,8005305.102,37,4941.672,35,31-Mar-2004,D.R. Tilley and J.H. Kelley and J.L. Godwin ...,2022-04-26
1431,52,52,Te,,NaN,,NaN,,0,,...,437,8157.3256,30539,103946723.408,340967,-49626.831,317609,31-JAN-2021,BALRAJ SINGH,2022-04-26
1432,52,53,Te,,NaN,,NaN,,0,,...,317,8186.8368,28573,104943304.516,322084,-52811.509,300020,1-Apr-2019,S. LALKOVSKI and J. TIMAR and Z. ELEKES,2022-04-26
1433,52,54,Te,,NaN,,NaN,,0,,...,103,8236.7682,9485,105937498.521,107934,-58219.759,100541,1-May-2007,D. DE FRENNE AND A. NEGRET,2022-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,116,176,Lv,,NaN,,NaN,,0,,...,1059,7116,3,292201969,819,188133.000,763,31-Jan-2019,BALRAJ SINGH,2022-04-26
3362,116,177,Lv,,NaN,,NaN,,0,,...,869,7111,2,293204583,553,190568.000,515,12-Aug-2005,M. GUPTA and THOMAS W. BURROWS,2022-04-26
3364,117,176,Ts,,NaN,,NaN,X,,,...,1090,7095,3,293208727,835,194428.000,778,1-August-2010,M. GUPTA,2022-04-26
3365,117,177,Ts,,NaN,,NaN,,0,,...,785,7092,2,294210840,637,196397.000,593,31-Jan-2019,BALRAJ SINGH,2022-04-26


In [3]:
df["energy"] = [float(e) * units.keV 
if not (e == '0' or e == ' ')
else "nan" for e in df["qa"]]
df["energy"]

27                 nan
29         91.8399 keV
1431     5096.3997 keV
1432     5069.1999 keV
1433     4290.2208 keV
             ...      
3361    10791.1897 keV
3362    10677.3271 keV
3364    11319.9999 keV
3365    11179.9999 keV
3366    11867.3266 keV
Name: energy, Length: 467, dtype: object

In [4]:
df["half_life_sec"] = [float(s) * units.second
if not (s == '0' or s == ' ')
else "nan" for s in df["half_life_sec"]]
df["half_life_sec"]

27      4.959105598602019e-21 s
29      8.190982317260066e-17 s
1431                  1.8e-08 s
1432                  6.2e-07 s
1433                    7e-05 s
                 ...           
3361                   0.0128 s
3362                    0.053 s
3364                    0.014 s
3365                    0.051 s
3366                  0.00058 s
Name: half_life_sec, Length: 467, dtype: object

In [5]:
df["z"] = [float(s) * units.g / units.mol
if not (s == '0' or s == ' ')
else "nan" for s in df["z"]]
df["z"]

27        4.0 g / mol
29        4.0 g / mol
1431     52.0 g / mol
1432     52.0 g / mol
1433     52.0 g / mol
            ...      
3361    116.0 g / mol
3362    116.0 g / mol
3364    117.0 g / mol
3365    117.0 g / mol
3366    118.0 g / mol
Name: z, Length: 467, dtype: object

By differentiating the equation in the definition of half life, we have for the power in watts emitted by 1 mole of a radioisotope with half-life in seconds $t$ and decay energy in joules $E$
\begin{equation}
P = \frac{E \ln(2)}{\ln(t)}
\end{equation}
Now we have for the specific power

In [6]:
def calc_spec_p(t, E, M):
  if "nan" in (t, E, M):
     return "nan"
  out = E * const.N_A*np.log(2) / (M * t) 
  return (out).to(units.W / units.g)

calc_spec_p(1 * units.second, 
100 * units.keV, 1 * units.g / units.mol)

<Quantity 6.68785359e+09 W / g>

In [7]:
df.iloc[:,16]

27      4.959105598602019e-21 s
29      8.190982317260066e-17 s
1431                  1.8e-08 s
1432                  6.2e-07 s
1433                    7e-05 s
                 ...           
3361                   0.0128 s
3362                    0.053 s
3364                    0.014 s
3365                    0.051 s
3366                  0.00058 s
Name: half_life_sec, Length: 467, dtype: object

In [8]:
df.iloc[:,0]

27        4.0 g / mol
29        4.0 g / mol
1431     52.0 g / mol
1432     52.0 g / mol
1433     52.0 g / mol
            ...      
3361    116.0 g / mol
3362    116.0 g / mol
3364    117.0 g / mol
3365    117.0 g / mol
3366    118.0 g / mol
Name: z, Length: 467, dtype: object

In [9]:
df.iloc[:,8]

27                 nan
29         91.8399 keV
1431     5096.3997 keV
1432     5069.1999 keV
1433     4290.2208 keV
             ...      
3361    10791.1897 keV
3362    10677.3271 keV
3364    11319.9999 keV
3365    11179.9999 keV
3366    11867.3266 keV
Name: energy, Length: 467, dtype: object

In [10]:
df["specific power (W/g)"] = [
calc_spec_p(df.iloc[row, 16],
df.iloc[row, 8], df.iloc[row, 0])
for row in range(df.shape[0])]

In [11]:
df["specific power (W/g)"]

27                               nan
29      1.8746585616197635e+25 W / g
1431     3.641450325139752e+17 W / g
1432    1.0515529393039166e+16 W / g
1433         78825188435745.53 W / g
                    ...             
3361         486058033434.0501 W / g
3362        116148992202.02902 W / g
3364        462188656899.85535 W / g
3365        125306188197.12259 W / g
3366        11596572594557.781 W / g
Name: specific power (W/g), Length: 467, dtype: object

In [12]:
df = df[df["specific power (W/g)"] != "nan"]

In [13]:
df

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift,energy,unc_e,...,binding,unc_ba,atomic_mass,unc_am,massexcess,unc_me,ENSDFpublicationcut-off,ENSDFauthors,Extraction_date,specific power (W/g)
29,4.0 g / mol,4,Be,,NaN,,NaN,,91.8399 keV,,...,7062.4356,44,8005305.102,37,4941.672,35,31-Mar-2004,D.R. Tilley and J.H. Kelley and J.L. Godwin ...,2022-04-26,1.8746585616197635e+25 W / g
1431,52.0 g / mol,52,Te,,NaN,,NaN,,5096.3997 keV,,...,8157.3256,30539,103946723.408,340967,-49626.831,317609,31-JAN-2021,BALRAJ SINGH,2022-04-26,3.641450325139752e+17 W / g
1432,52.0 g / mol,53,Te,,NaN,,NaN,,5069.1999 keV,,...,8186.8368,28573,104943304.516,322084,-52811.509,300020,1-Apr-2019,S. LALKOVSKI and J. TIMAR and Z. ELEKES,2022-04-26,1.0515529393039166e+16 W / g
1433,52.0 g / mol,54,Te,,NaN,,NaN,,4290.2208 keV,,...,8236.7682,9485,105937498.521,107934,-58219.759,100541,1-May-2007,D. DE FRENNE AND A. NEGRET,2022-04-26,78825188435745.53 W / g
1434,52.0 g / mol,55,Te,,NaN,,NaN,,4010.3533 keV,,...,8258,1,106934882,108,-60657.000,101,1-Mar-2008,Jean Blachot,2022-04-26,1663812389904.8296 W / g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,116.0 g / mol,176,Lv,,NaN,,NaN,,10791.1897 keV,,...,7116,3,292201969,819,188133.000,763,31-Jan-2019,BALRAJ SINGH,2022-04-26,486058033434.0501 W / g
3362,116.0 g / mol,177,Lv,,NaN,,NaN,,10677.3271 keV,,...,7111,2,293204583,553,190568.000,515,12-Aug-2005,M. GUPTA and THOMAS W. BURROWS,2022-04-26,116148992202.02902 W / g
3364,117.0 g / mol,176,Ts,,NaN,,NaN,X,11319.9999 keV,,...,7095,3,293208727,835,194428.000,778,1-August-2010,M. GUPTA,2022-04-26,462188656899.85535 W / g
3365,117.0 g / mol,177,Ts,,NaN,,NaN,,11179.9999 keV,,...,7092,2,294210840,637,196397.000,593,31-Jan-2019,BALRAJ SINGH,2022-04-26,125306188197.12259 W / g


In [14]:
df["half_life_sec"]

29      8.190982317260066e-17 s
1431                  1.8e-08 s
1432                  6.2e-07 s
1433                    7e-05 s
1434                   0.0031 s
                 ...           
3361                   0.0128 s
3362                    0.053 s
3364                    0.014 s
3365                    0.051 s
3366                  0.00058 s
Name: half_life_sec, Length: 453, dtype: object

In [15]:
df["half_life_sec"] =[float(t / units.second)
for t in df["half_life_sec"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
month = (units.year / 13).to(units.second) / units.second
century = (units.year * 100).to(units.second) / units.second
df = df[df["half_life_sec"] > month]
df = df[df["half_life_sec"] < century]

In [17]:
df

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift,energy,unc_e,...,binding,unc_ba,atomic_mass,unc_am,massexcess,unc_me,ENSDFpublicationcut-off,ENSDFauthors,Extraction_date,specific power (W/g)
1772,60.0 g / mol,84,Nd,4.9421,0.0027,23.798,19,,1901.2829 keV,,...,8326.9237,87,143910092.798,1346,-83748.028,1255,1-Dec-2000,A.A. Sonzogni,2022-04-26,2.932594904082594e-14 W / g
1847,62.0 g / mol,84,Sm,4.9808,0.0035,,NaN,,2528.7792 keV,,...,8293.8581,209,145913046.835,3269,-80996.360,3045,14-Jul-2016,Yu. KHAZOV and A. RODIONOV AND G. SHULYAK,2022-04-26,1.271166311097269e-06 W / g
1848,62.0 g / mol,85,Sm,4.9892,0.0035,15.00,14,,2311.3466 keV,,...,8280.5734,86,146914904.401,1354,-79266.048,1262,9-Mar-2022,N. Nica and B. Singh,2022-04-26,7.363185300713775e-10 W / g
1849,62.0 g / mol,86,Sm,5.0042,0.0034,11.25,9,,1987.0458 keV,,...,8279.6326,84,147914829.233,1337,-79336.067,1246,1-Oct-2013,N. NICA,2022-04-26,9.70309344810548e-15 W / g
1890,63.0 g / mol,88,Eu,5.0522,0.0046,47.81,6,,1963.97 keV,,...,8239.2998,77,150919856.606,1251,-74653.098,1166,20-Nov-2008,BALRAJ SINGH,2022-04-26,3.8863100424198563e-17 W / g
1923,64.0 g / mol,84,Gd,5.0080,0.0171,,NaN,,3271.2911 keV,,...,8248.3398,99,147918121.414,1566,-76269.419,1460,1-Oct-2013,N. NICA,2022-04-26,1.5235674422540857 W / g
1925,64.0 g / mol,86,Gd,5.0342,0.0159,,NaN,,2807.3935 keV,,...,8242.6103,404,149918663.949,6500,-75764.051,6055,1-Apr-2013,S. K. Basu and A. A. Sonzogni,2022-04-26,5.193526490707533e-05 W / g
1927,64.0 g / mol,88,Gd,5.0774,0.0048,0.20,3,,2203.8471 keV,,...,8233.4042,66,151919798.414,1081,-74707.304,1007,31-Aug-2013,M. J. MARTIN,2022-04-26,6.757247045670047e-13 W / g
2001,66.0 g / mol,88,Dy,5.1241,0.2618,,NaN,,2945.1204 keV,,...,8193.1305,483,153924428.92,7977,-70394.015,7431,1-May-2008,C. W. Reich,2022-04-26,3.1523173317589436e-05 W / g
2237,72.0 g / mol,102,Hf,5.3201,0.0035,0.16,12,,2494.4298 keV,,...,8068.5341,130,173940048.377,2425,-55844.583,2259,1-Nov-1998,E. BROWNE and HUO JUNDE,2022-04-26,3.671139268421588e-14 W / g


In [18]:
df["specific power (W/g)"] = [float(e * units.g / units.W)
for e in df["specific power (W/g)"]]
df["z"] = [int(z * units.mol / units.g) for z in df["z"]]
df["Half-life (s)"] = df["half_life_sec"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [33]:
df["A"] = df["z"] + df["n"]

In [34]:
fig = px.scatter(df, x = "Half-life (s)", y = "specific power (W/g)",
           hover_data = ["A", "symbol"], log_x = True, log_y = True)
fig.show()

Which appears to agree with https://en.wikipedia.org/wiki/Geiger%E2%80%93Nuttall_law